# Install Required Libraries

In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets einops

# Log in to WandB to track finetuning metrics
# Log in to Hugging Face

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!pip install wandb
!wandb login

# Load & Quantize the model

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map={"":0})
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Import IA3 Config

In [ ]:
from peft import IA3Config, get_peft_model

config = IA3Config(
    peft_type="IA3",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "v_proj"],
    feedforward_modules=["v_proj"],
)

model = get_peft_model(model, config)

In [ ]:
print(model)

PeftModelForCausalLM(
  (base_model): IA3Model(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 4096x1])
              )
              (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (v_proj): Linear(
                in_features=4096, out_features=4096, bias=False
                (ia3_l): ParameterDict(  (default): Parameter containing: [torch.FloatTensor of size 1x4096])
              )
              (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
              (rotary_emb): LlamaRotaryEmbedding()
            )
            (mlp): LlamaMLP(
              (gate_proj): Linear(in

# Load and preprocessing the dataset

In [ ]:
from datasets import load_dataset

# Split the data into 10% test, 10% validation, 80% training
train_data = load_dataset("Hieu-Pham/cooking_squad_splitted", split='train')
val_data = load_dataset("Hieu-Pham/cooking_squad_splitted", split='validation')

def group_train_data(dataset):

    grouped = []

    for row in dataset:
      answers = row['answers']
      input = 'Question: ' + row['question'] + ' Context: ' + row['context'] + ' Answer: ' + answers['text']
      grouped.append(input)

    return grouped

input_column = group_train_data(train_data)
train_data = train_data.add_column('Inputs', input_column)

train_data = train_data.map(
    lambda row: tokenizer(row["Inputs"], truncation=True),
    batched=True,
    remove_columns=train_data.column_names,
)

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/1793 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [ ]:
# Preprocessing test and validation dataset

def group_test_data(dataset):

    grouped = []

    for row in dataset:
      answers = row['answers']
      input = 'Question: ' + row['question'] + ' Context: ' + row['context']
      grouped.append(input)

    return grouped

# Validation dataset
output_column = group_test_data(val_data)
val_data = val_data.add_column('Outputs', output_column)

val_data = val_data.map(
    lambda row: tokenizer(row["Outputs"], truncation=True),
    batched=True,
    remove_columns=val_data.column_names,
)

Map:   0%|          | 0/224 [00:00<?, ? examples/s]

# Training Configurations and Start Training

In [ ]:
import transformers

training_args = transformers.TrainingArguments(
    output_dir="Llama2-7B-IA3-cooking-text-gen",
    evaluation_strategy="steps",
    learning_rate=2e-4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    weight_decay=0.01,
    save_steps=10,
    logging_steps=20,
    max_grad_norm=0.3,
    max_steps=200,
    warmup_ratio=0.03,
    fp16=True,
    optim="paged_adamw_8bit",
    push_to_hub=True,
)


In [ ]:
# tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings
trainer.train()

wandb: Currently logged in as: hieupham. Use `wandb login --relogin` to force relogin


You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
20,1.487300,1.500468
40,1.441000,1.470042
60,1.436800,1.452831
80,1.411800,1.438660
100,1.386100,1.426740
120,1.377400,1.415866
140,1.362600,1.407074
160,1.359000,1.401021
180,1.346100,1.397552
200,1.347500,1.396376


TrainOutput(global_step=200, training_loss=1.3955546092987061, metrics={'train_runtime': 279.7295, 'train_samples_per_second': 11.44, 'train_steps_per_second': 0.715, 'total_flos': 3.559575369842688e+16, 'train_loss': 1.3955546092987061, 'epoch': 1.78})